In [1]:
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import fftpack
import numpy as np
import pandas as pd
import sounddevice as sd

In [2]:
data = scipy.io.loadmat('bancovozes.mat')

dbkids = data['dbkids']
dbmen = data['dbmen']
dbwomen = data['dbwomen']

vozes = pd.DataFrame(np.hstack((dbkids[:][3000:5000], dbwomen[:][3000:5000], dbmen[:][3000:5000])))

l_kids = len(dbkids.T)
l_men = len(dbmen.T)
l_women = len(dbwomen.T)

y_dbkids  = np.full((1, l_kids) , 0)
y_dbmen   = np.full((1, l_men)  , 1)
y_dbwomen = np.full((1, l_women), 3)

y = np.concatenate((y_dbkids, y_dbmen, y_dbwomen),axis=1)

In [3]:
fft_vozes = []

for n in range(len(vozes.T)):
    fft_data = np.abs(fftpack.fft(vozes[n][:]))
    max_ = np.amax(fft_data)
    norm_fft_data = fft_data/max_
    fft_vozes.append(norm_fft_data)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(fft_vozes,y.T, test_size=0.2)

In [16]:
# Define a RNA com solver, numero de neuronios, ...
clf = MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=5,
              learning_rate='constant', learning_rate_init=0.01,
              max_iter=1000, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5,
              shuffle=True, solver='adam', tol=0.0001,
validation_fraction=0.1, verbose=False, warm_start=False)

In [22]:
# Treina a RNA
info = clf.fit(X_train, y_train)

/home/palmdev/anaconda3/envs/palmdev/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
# Valores da saida preditos
y_predicted = clf.predict(X_test)

In [24]:
100*np.count_nonzero(y_predicted-y_test.T)/len(y_test)

29.09090909090909

In [25]:
print(y_predicted-y_test.T)

[[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0 -2  0
   0 -1  0 -1  0  0  0  1  0  0  0  0  0  0  0 -1 -2  0  0  0  0  1  0  1
   0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  1  0  0  0 -1  0 -3 -1  0
   0 -1 -1  0  0  1  1  0  0 -2  0  0 -1  0  0  0  0  0  0 -3  0  0  0  0
   0  2  0  0  0 -1 -2  0  2 -2  0  0  0  0  1 -1  0  1  0  3  0 -3  0 -1
   0  0  0  0  0  1  1  0 -1 -1  0  0  0  0 -3  0  3  1  0 -1 -2  0  0 -1
   0  3  0  0  0  1  0  0  0  1  0  0  0  0  0  1  0  0  2  0  0  0  0  0
   0  0  0 -1  1  0  1  0  1  0 -1  1  0  0  0 -1  0  1  0 -1  1  2 -2  0
   0  0 -3  1  0  0  0 -1  0  0  0  1  0  0  0 -1  0  0 -1  0  0  0  0  0
   0  1  0  1  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0 -1  0  0  0 -1
  -3  0  0 -1  0  0  0  0  0  0  1 -1  0 -1 -2  0 -1  0  0 -2  0  0  0  0
   0  0  0  0  0  0  1  1  0  0 -1  0  0  0  0  0  0  1 -2  1  0  0 -1  0
   0  0  0  0  0  0  0  0  0  1  0  0  0 -2  1  0 -2  1  0  0  0  0  0  0
   0  0  0 -1  0  0  0  0  0  0  0  0 